# **Maestría en Inteligencia Artificial Aplicada**
## **Curso: Procesamiento de Lenguaje Natural (NLP)**
### Tecnológico de Monterrey
### Prof Luis Eduardo Falcón Morales

## **Adtividad Semanas 05**

### **Modelos Embebidos: Caso Amazon/Yelp/IMDb**

#### Equipo 15

**Nombres y matrículas de los integrantes del equipo:**

* Javier de Alba Pérez $\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,$ A01226046
* Irvin Gomez Esquivel $\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,$ A00361034
* Daniel Camacho $\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,$ A01793555
* Manuel Gerardo Licera Aguirre $\,\,\,\,\,\,\,\,\,\,\,$ A00947315
* Andrés Eduardo Figueroa García $\,\,\,\,\,\,$ A01378536

## **Instrucciones**

En la actividad de esta semana trabajarás en equipos con el modelo de vectores continuos/embebidos FastText,
es decir, el modelo desarrollado por Facebook en 2016.

Una manera de trabajar con estos modelos pre-entrenados, es generando el vocabulario a partir de tu conjunto
de datos de entrenamiento. Para cada palabra de tu vocabulario, podrás sustituirlo por su correspondiente
vector continuo. En caso de que no exista el vector para una palabra en particular, se puede eliminar dicha
palabra, o bien sustituirla por el vector continuo más cercano. En esta actividad deberás aplicar esta segunda
opción. Existen diversas propuestas para utilizar dichos vectores continuos como entrada para modelos de
aprendizaje automático. En particular, en esta actividad cada enunciado será sustituido por el vector promedio
de todos los tokens que lo forman.

## **Librerias**

In [2]:
import pandas as pd  
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
import string

## **Pregunta - 1:**

Descarga los 3 archivos de Canvas. En particular, el archivo de datos de IMDb ya no requiere
transformarse para obtener sus 1000 registros. Al cargar los datos de los tres archivos deberás
tener un DataFrame de Pandas de 3000 registros, con sus etiquetas. Los archivos los encuentras en
Canvas y se llaman: amazon5.txt, imdb5.txt, yelp5.txt.

In [3]:
dfa = pd.read_csv('amazon5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')
dfi = pd.read_csv('imdb5.txt', sep='   ', names=['review','label'], header=None, encoding='utf-8')
dfy = pd.read_csv('yelp5.txt', sep='\t', names=['review','label'], header=None, encoding='utf-8')


print('Total de registros de Amazon:',dfa.shape)
print('Total de registros de IMBD:',dfi.shape)
print('Total de registros de Yelp:',dfy.shape)

Total de registros de Amazon: (1000, 2)
Total de registros de IMBD: (1000, 2)
Total de registros de Yelp: (1000, 2)


C:\Users\irvin\AppData\Local\Temp\ipykernel_91112\399887763.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dfi = pd.read_csv('imdb5.txt', sep='   ', names=['review','label'], header=None, encoding='utf-8')


In [4]:
with open('imdb5.txt') as f:
    lines = f.readlines()

texto_limpio = []

for linea in lines:
    linea_limpia = linea.strip().replace('\t', ' ')
    texto_limpio.append(linea_limpia)

In [5]:
data = {'review': [], 'label': []}
for line in texto_limpio:
    line = line.strip()
    review, label = line.rsplit(' ', 1)
    data['review'].append(review)
    data['label'].append(label)

dfi_limpio = pd.DataFrame(data)

In [6]:
df = pd.concat([dfa, dfi, dfy], ignore_index=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  3000 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [8]:
df.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [9]:
df.dtypes

review    object
label      int64
dtype: object

In [10]:
# df[1000:1030]
print (df)

                                                 review  label
0     So there is no way for me to plug it in here i...      0
1                           Good case, Excellent value.      1
2                                Great for the jawbone.      1
3     Tied to charger for conversations lasting more...      0
4                                     The mic is great.      1
...                                                 ...    ...
2995  I think food should have flavor and texture an...      0
2996                           Appetite instantly gone.      0
2997  Overall I was not impressed and would not go b...      0
2998  The whole experience was underwhelming, and I ...      0
2999  Then, as if I hadn't wasted enough of my life ...      0

[3000 rows x 2 columns]


## **Pregunta - 2:**

Realiza de nuevo un proceso de limpieza. Aplica el preprocesamiento que consideres adecuado, sin
embargo, deberás aplicar necesariamente alguna de las técnicas de lematización. Como aplicaremos
modelos embebidos pre-entrenados, queremos palabras lo más cercanas a las existentes en un
idioma, inglés en este caso. Aplica y justifica cualquier otro proceso de limpieza que consideres
adecuado. Recuerda que en esta actividad se usarán vectores embebidos para un problema de
clasificación, por lo que deberás tomar de acuerdo a este contexto. Justifica todas las
transformaciones que se apliquen.

In [11]:
nltk.download('stopwords') 

mystopwords = [x for x in stopwords.words('english')]

X = df.review

def clean_tok(doc):
  #carateres alfabéticos, primero remplazamos signos de puntuación por espacios
  #luego removemos espacios adicionales
  tmpdoc = re.sub(r'[^\w\s]', ' ', doc)
  tmpdoc = re.sub(r'[\s{2,}]', ' ', tmpdoc)

  #Minusculas
  tmpdoc = tmpdoc.lower()

  #Tokenizacion
  tokens = []
  tokens = re.findall(r'\b[a-z]+\b', tmpdoc)

  #Longitud mayor a 1
  tokens = [i for i in tokens if len(i)>1]

  #Stopwords
  tokens = [i for i in tokens if i not in mystopwords]
  return tokens



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\irvin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
Xcleantok = [clean_tok(x) for x in X] 

In [13]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

def clean_doc(doc):
  
  # Primero vamos a lematizar
  lemmatizer = WordNetLemmatizer()
  tmptokens = []
  for x in doc:
    tmptokens.append(lemmatizer.lemmatize(x))

  # Y también vamos a remover tokens duplicados
  tokens = []
  [tokens.append(x) for x in tmptokens if x not in tokens]

  return tokens

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\irvin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
Xclean = [clean_doc(x) for x in Xcleantok]  

## **Pregunta - 3:**

Llamar Xclean a los comentarios procesados y Y a las etiquetas. Realicemos una partición aleatoria
con los mismos porcentajes de la práctica pasada para poder comparar dichos resultados con los de
esta actividad, a saber, 70%, 15% y 15%, para entrenamiento, validación y prueba,
respectivamente. Verifica que obtienes 2100 registros de entrenamiento y 450 para cada uno de
validación y prueba.

In [21]:
Y = df.label

# verificando que tenemos la dimensiones esperadas.
assert Y.shape == (3000,)


In [22]:
from sklearn.model_selection import train_test_split

x_train, x_val_and_test, y_train, y_val_and_test = train_test_split(Xclean, Y, train_size=.70, shuffle=True, random_state=6) 
x_val, x_test, y_val, y_test = train_test_split(x_val_and_test, y_val_and_test, test_size=.50, shuffle=True, random_state=11)

print('X,y Train:', len(x_train), len(y_train))    
print('X,y Val:', len(x_val), len(y_val))
print('X,y Test', len(x_test), len(y_test))

X,y Train: 2100 2100
X,y Val: 450 450
X,y Test 450 450


## **Pregunta - 4:**

Usando el conjunto de entrenamiento genera un vocabulario que no sea mayor a 1500 palabras, ni
menor a 1000. ¿Por qué es importante acotar un vocabulario inferior y superiormente? ¿Por qué
debe usarse solamente el conjunto de entrenamiento para generar el diccionario? Con este
vocabulario que obtienes, filtra los conjuntos de entrenamiento, validación y prueba, de esta
manera todos los comentarios usarán solamente palabras válidas de acuerdo a este vocabulario.
Indica el tamaño del vocabulario obtenido.

Hasta este punto básicamente has realizado transformaciones muy análogas a las de la semana
pasada y que son válidas para muchos de los procesos dentro del análisis de textos. En dado caso
comenta con tus compañeros de equipo qué diferencias has observado. Veamos ahora la diferencia
con respecto a las matrices Tf-idf que aplicaste la semana pasada, con respecto a los vectores preentrenados
embebidos.

In [ ]:
None

## **Pregunta - 5:**

Utilizarás los vectores embebidos FastText preentrenados por Facebook.
a. Incluye una tabla comparativa de pros y contras entre los modelos FastText, word2vec de
Google y Glove de Stanford. Puedes consultar sus páginas correspondientes:
https://fasttext.cc/
https://code.google.com/archive/p/word2vec/
https://nlp.stanford.edu/projects/glove/

In [ ]:
None

## **Pregunta - 6:**

Utiliza el modelo FastText de vectores embebidos pre-entrenados de dimensión 300 para generar
un nuevo diccionario clave-valor, donde la “clave” será cada token o palabra de tu vocabulario y el
“valor” será su vector embebido de dimensión 300. Este diccionario deberá ser del mismo tamaño
que el vocabulario previo que hayas construido previamente.
https://fasttext.cc/docs/en/crawl-vectors.html

NOTA: Debido a la cantidad de recursos computacionales que demanda cargar los vectores
FastText (son 2 millones de vectores), es recomendable que una vez que generes el nuevo
vocabulario de vectores embebidos, guardes dicho diccionario en un archivo (pickle, npz o el que
consideres más adecuado). Una vez realizado lo anterior, puedes borrar la variable de FastText
para liberar memoria RAM. De esta manera, ya tienes tu vocabulario de vectores embebidos de
acuerdo a los tokens que consideras más adecuados para tu problema y puedes usarlo rápidamente
cuando lo necesites. En dado caso apóyense entre los miembros del equipo de tener dificultades
para generar el vocabulario y por mientras puedes usar el archivo del vocabulario que alguno haya
generado

In [ ]:
None

## **Pregunta - 7:**

Una manera de utilizar los vectores embebidos con modelos de aprendizaje automático en
documentos de texto, es asignar a cada comentario filtrado el vector embebido de dimensión 300
que resulta de promediar todos sus tokens. Así, en este ejercicio deberás generar los arreglos
correspondientes para los conjuntos de entrenamiento, validación y prueba. Los llamaremos
trainEmb, valEmb y testEmb, respectivamente. ¿Cuáles son sus dimensiones? ¿Se podrían usar para
su representación matrices dispersas (sparse matrices) como en el caso de la matriz Tf-idf?
Responde a dichas preguntas.

In [ ]:
None

## **Pregunta - 8:**

Utiliza los modelos de regresión lineal y bosque aleatorio (random forest) y encuentra sus
desempeños (accuracy). Compara los resultados con los de la semana anterior.

In [ ]:
None

## **Pregunta - 9:**

Obtener la matriz de confusión e interpretar sus valores.

In [ ]:
None

## **Pregunta - 10:**

Comenta con tus compañeros de equipo los pasos realizados en esta actividad e incluyan sus
conclusiones finales.

In [ ]:
None

# **FIN DE LA ACTIVIDAD DE LA SEMANA 5**